In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
from natsort import natsorted
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import torchvision.datasets as datasets

# Define transforms for the dataset
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [31]:
nspath = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US'
all_ns_path = glob(os.path.join(nspath,'*','*.png')) # list of all images
all_ns_path = natsorted(all_ns_path)
non_empty_images_df = pd.read_excel(r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\ns_train_image_path.xlsx")
non_empty_images_df = non_empty_images_df.drop('Unnamed: 0.1',axis=1)
non_empty_images_df.iloc[:,1] = non_empty_images_df.iloc[:,1].str.replace('registrated', 'registered')
non_empty_images_df
# non_empty_images = list(non_empty_images_df.iloc[:,1])
# non_empty_images = natsorted(non_empty_images)
# empty_images = set(all_ns_path) - set (non_empty_images)
# print(len(empty_images))

AttributeError: Can only use .str accessor with string values!

In [39]:

non_empty_images_df

,Unnamed: 0
0,0
1,1
2,2
3,3
4,4
...,...
79266,79266
79267,79267
79268,79268
79269,79269


In [38]:
non_empty_images_df.to_excel(r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\ns_train_image_path.xlsx")

In [ ]:
#define dataset and dataloaders
train_dataset = ImageFolder(root='path/to/train/folder', transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = ImageFolder(root='path/to/val/folder', transform=val_transform)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# #define pre-trained resNet-18 model
# model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)  #replace/edit output layer with a new linear layer for binary classification- blank or not blank
model = EfficientNet.from_pretrained('efficientnetv2-s')

# Modify the last layer to output a single binary classification output
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, 1)

#define loss function, optimizer and device
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

#training loop
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_loss += loss.item()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_loss = train_loss / len(train_dataset)
    train_accuracy = train_correct / train_total

    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_loss += loss.item()
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_loss = val_loss / len(val_dataset)
    val_accuracy = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
